In [1]:
import cv2
import os
from ultralytics import YOLO
import numpy as np
import torch
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm


In [4]:
# specify image dir and mask save directory
save_dir = '/home/logan/Desktop/Multys_something/dataset/sequences/00/mask_3'
img_dir = '/home/logan/Desktop/Multys_something/dataset/sequences/00/image_3'

# create model and predict 
model = YOLO('yolov8s-seg.pt')
kernel = np.ones((7, 7), np.uint8) # dialation kernel

# get all images
image_list = glob(os.path.join(img_dir, '*'))
image_list.sort()

for img_file in tqdm(image_list):
    # show images
    img = cv2.imread(img_file)
    img_name = os.path.basename(img_file)
    # print(img_name)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # plt.imshow(rgb_img)
    # print(img.shape)

    # detect segment
    # results = model.predict(source=img.copy(), save=True, save_txt=False, stream=True, verbose=False)
    results = model.predict(source=img.copy(), verbose=False)

    # create binary mask and save it 
    for result in results:
        if result.masks is None:
            object_mask = np.zeros_like(img)
            cv2.imwrite(os.path.join(save_dir, img_name), object_mask)
            continue
        # get array results
        masks = result.masks.data
        boxes = result.boxes.data
        # extract classes
        clss = boxes[:, 5]
        # print(clss)
        # get indices of results where class is 0 (people in COCO)
        # people_indices = torch.where(clss in [0,1,2,3,4,5,6,7,8])
        indixes = torch.where(clss == 0)[0]
        for idx in [1,2,3,4,5,6,7,8]:
            car_indices = torch.where(clss == idx)
            indixes = torch.cat((indixes, car_indices[0]))
        # people_indices = torch.where(clss == 0)
        # car_indices = torch.where(clss == 2)
        # indixes = torch.cat((people_indices[0], car_indices[0]))
        # use these indices to extract the relevant masks
        object_masks = masks[indixes]
        # scale for visualizing results
        object_mask = torch.any(object_masks, dim=0).int()

        ## dilation
        # object_mask = object_mask.cpu().numpy().astype('uint8') * 255
        object_mask = object_mask.cpu().numpy().astype('uint8')
        object_mask = cv2.dilate(object_mask, kernel)  * 255
        
        # resize mask to the same as the input image
        object_mask = cv2.resize(object_mask, (img.shape[1], img.shape[0]))
        # save to file
        cv2.imwrite(os.path.join(save_dir, img_name), object_mask)
        # print(object_mask.shape)
        # plt.figure()
        # plt.imshow(object_mask)


  0%|          | 0/972 [00:00<?, ?it/s]

100%|██████████| 972/972 [00:12<00:00, 77.35it/s]


In [ ]:
# use IOPaint

! iopaint run --model=lama --device=cuda --image=/home/logan/Desktop/Multys_something/dataset/image --mask=/home/logan/Desktop/Multys_something/dataset/mask --output=/home/logan/Desktop/Multys_something/dataset/output

/bin/bash: iopaint: command not found
